## DIA 136: Convertir el Excel con Gráficos y Estadísticas en un PDF Autogenerado 

Hoy vas a aprender a convertir automáticamente el Excel con gráficos y estadísticas en un archivo PDF, listo para descarga, presentación o impresión. Esto facilita:

📤 Compartir informes visuales con colegas

🖨️ Imprimir reportes con gráficos ya formateados

📁 Almacenar versiones estáticas del historial

❗ Nota importante:
Python no convierte directamente Excel a PDF con gráficos incrustados como lo haría Excel nativo. Las mejores soluciones son:

Usar Excel COM con pywin32 (solo en Windows con Excel instalado)

O bien, recrear el PDF directamente desde Python (usaremos esta opción)

✅ Opción recomendada: recrear el contenido en PDF con reportlab y matplotlib
🧩 1. Instalar librerías necesarias
bash
Copiar
Editar
pip install reportlab matplotlib pandas
🧩 2. Generar el PDF con resumen y gráficos
python
Copiar
Editar
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Image, Table
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib.pagesizes import A4
from matplotlib import pyplot as plt
from io import BytesIO
import pandas as pd

@app.route('/exportar-reporte-pdf')
@login_required
def exportar_pdf():
    # Obtener los reportes del usuario
    reportes = Reporte.query.filter_by(user_id=current_user.id).all()
    df = pd.DataFrame([{
        'Fecha': r.fecha_generado.strftime('%Y-%m-%d'),
        'Tipo': r.tipo,
        'Categoría': r.categoria or 'Sin categoría'
    } for r in reportes])

    # === Gráfico de Categorías ===
    fig1, ax1 = plt.subplots(figsize=(6, 3))
    df['Categoría'].value_counts().plot(kind='bar', color='mediumseagreen', ax=ax1)
    ax1.set_title('Reportes por Categoría')
    ax1.set_ylabel('Cantidad')
    img1 = BytesIO()
    plt.tight_layout()
    fig1.savefig(img1, format='PNG')
    plt.close(fig1)
    img1.seek(0)

    # === Gráfico de Tendencia ===
    fig2, ax2 = plt.subplots(figsize=(6, 3))
    df_fecha = df['Fecha'].value_counts().sort_index()
    df_fecha.plot(kind='line', ax=ax2, color='royalblue')
    ax2.set_title('Reportes por Día')
    ax2.set_xlabel('Fecha')
    ax2.set_ylabel('Cantidad')
    img2 = BytesIO()
    plt.tight_layout()
    fig2.savefig(img2, format='PNG')
    plt.close(fig2)
    img2.seek(0)

    # === Crear PDF ===
    buffer = BytesIO()
    doc = SimpleDocTemplate(buffer, pagesize=A4)
    styles = getSampleStyleSheet()
    story = []

    story.append(Paragraph("📄 Informe de Reportes", styles['Title']))
    story.append(Spacer(1, 12))
    story.append(Paragraph(f"Total reportes: {len(df)}", styles['Normal']))
    story.append(Paragraph(f"Categorías únicas: {df['Categoría'].nunique()}", styles['Normal']))
    story.append(Spacer(1, 12))

    story.append(Paragraph("📊 Gráfico por Categoría", styles['Heading2']))
    story.append(Image(img1, width=400, height=200))
    story.append(Spacer(1, 12))

    story.append(Paragraph("📈 Gráfico de Tendencia Diaria", styles['Heading2']))
    story.append(Image(img2, width=400, height=200))

    doc.build(story)
    buffer.seek(0)

    return send_file(buffer, as_attachment=True, download_name='reporte_visual.pdf', mimetype='application/pdf')
🧩 3. Agrega botón en tu interfaz (ej: estadisticas_reportes.html)
html
Copiar
Editar
<a href="{{ url_for('exportar_pdf') }}">
  <button>📄 Descargar PDF con Gráficos</button>
</a>